# 階層クラスタリング

このノートブックでは、階層クラスタリングの実装と可視化を学習します。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)

# データの生成
X, y_true = make_blobs(n_samples=100, centers=3, random_state=42)
X_scaled = StandardScaler().fit_transform(X)

# 階層クラスタリングの実装
def hierarchical_clustering_manual(X, n_clusters):
    n_samples = X.shape[0]
    
    # 初期化：各点を個別のクラスタとする
    clusters = [[i] for i in range(n_samples)]
    distances = pdist(X)
    
    # 距離行列の作成
    dist_matrix = np.zeros((n_samples, n_samples))
    idx = 0
    for i in range(n_samples):
        for j in range(i+1, n_samples):
            dist_matrix[i, j] = distances[idx]
            dist_matrix[j, i] = distances[idx]
            idx += 1
    
    # クラスタの結合
    while len(clusters) > n_clusters:
        min_dist = float('inf')
        merge_idx = (0, 0)
        
        # 最も近いクラスタペアを見つける
        for i in range(len(clusters)):
            for j in range(i+1, len(clusters)):
                # クラスタ間の最小距離を計算
                cluster_dist = min([dist_matrix[a, b] for a in clusters[i] for b in clusters[j]])
                if cluster_dist < min_dist:
                    min_dist = cluster_dist
                    merge_idx = (i, j)
        
        # クラスタを結合
        i, j = merge_idx
        clusters[i].extend(clusters[j])
        clusters.pop(j)
    
    # ラベルの作成
    labels = np.zeros(n_samples)
    for cluster_id, cluster in enumerate(clusters):
        for point_id in cluster:
            labels[point_id] = cluster_id
    
    return labels

# 手動実装とscikit-learnの比較
n_clusters = 3
labels_manual = hierarchical_clustering_manual(X_scaled, n_clusters)
hierarchical_sklearn = AgglomerativeClustering(n_clusters=n_clusters)
labels_sklearn = hierarchical_sklearn.fit_predict(X_scaled)

# 可視化
plt.figure(figsize=(15, 5))

# 元データ
plt.subplot(1, 3, 1)
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y_true, cmap='viridis', alpha=0.7)
plt.title('True Clusters')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# 手動実装
plt.subplot(1, 3, 2)
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels_manual, cmap='viridis', alpha=0.7)
plt.title('Manual Hierarchical')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# scikit-learn
plt.subplot(1, 3, 3)
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels_sklearn, cmap='viridis', alpha=0.7)
plt.title('scikit-learn Hierarchical')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

plt.tight_layout()
plt.show()

print("=== 階層クラスタリングの結果 ===")
print(f"手動実装のラベル: {labels_manual[:10]}...")
print(f"scikit-learnのラベル: {labels_sklearn[:10]}...")
print(f"クラスタ数: {len(np.unique(labels_manual))}")
